### Сторонние библиотеки использовать нельзя

### Задача 0 [Библиотека] (0.15 балла)  

**Условие:** 


В библиотеке хранятся книги и журналы. У каждой сущности есть общие характеристики, такие как: название, автор, жанр, число страниц, формат страниц, индекс редкости (от 1 до 10) и текст. Также у разных сущностей могут быть свои атрибуты. Хочется все редкие издания (индекс 9 или 10) дополнительно сохранять в некое хранилище (пусть json-файл), а также хочется понимать какую площадь занимает издание, если разложить все его страницы на полу.     


**Комментарий:**

Это задача с семинара на организацию иерархии классов. Идея в том, что нужно разделять сущности в зависимости от их применения. Например, есть книга как некий абстрактный объект, а есть библиотечная книга, у которой есть свои особенности. Также для сохранения книг в json нужно использвать классы-примеси.


Иерархия классов:

In [10]:
import datetime

In [1]:
PAGES_FORMAT = {
    'A1': (2048, 1024),
    'A2': (1024, 512),
    'A3': (512, 256),
    'A4': (297, 210),
}

GENRES = (
    "Роман",
    "Приключение",
    "Драма"
)

MAX_RARITY_IND = 10

In [2]:
class ValueFromSet:
    def __init__(self, name, values):
        self.name = name
        self._values = values
        
    def __set__(self, instance, value):
        if value not in self._values:
            raise ValueError("Неизвестное значение для {}: {}, должно быть одно из: {}"
                             .format(self.name, value, ", ".join(self._values)))
        instance.__dict__[self.name] = value 
        
    def __get__(self, instance, owner):
        return instance.__dict__[self.name]

In [3]:
class BoundedValue:
    def __init__(self, name, _min, _max=None):
        self._name = name
        self._min = _min
        self._max = _max
        
    def __set__(self, instance, value):
        if self._min is not None and self._max is not None and not self._min <= value < self._max:
            raise ValueError("Значение для {} должно быть от {} (вкл.) до {} (не вкл.)"
                             .format(self._name, self._min, self._max))
        elif self._min is not None and not value >= self._min:
            raise ValueError("Значение для {} должно быть больше или равно чем {}"
                             .format(self._name, self._min))
        elif self._max is not None and not value < self._max:
            raise ValueError("Значение для {} должно быть меньше {}"
                 .format(self._name, self._max))
            
        instance.__dict__[self._name] = value 
            
    def __get__(self, instance, owner):
        return instance.__dict__[self._name]

In [4]:
class NonNegative(BoundedValue):
    def __init__(self, name):
        super(NonNegative, self).__init__(name, 0)

In [5]:
class IntegerBounded(BoundedValue):
    def __init__(self, name, min_, max_):
        super().__init__(name, min_, max_)
    
    def __set__(self, instance, value):
        if not type(value) == int:
            raise ValueError("Значение для {} должно быть целочисленным". format(self.name))
        super(IntegerBounded, self).__set__(instance, value)

In [6]:
class DatetimeType:
    def __init__(self, name):
        self._name = name
    
    def __set__(self, instance, value):
        if not isinstance(value, datetime.date):
            raise ValueError("Неправильный тип {}, создайте дату как datetime.date(y,m,d)".format(self._name))
        instance.__dict__[self._name] = value 
        
    def __get__(self, instance, owner):
        return instance.__dict__[self._name]       

In [7]:
class BooleanType:
    def __init__(self, name):
        self._name = name
        
    def __set__(self, instance, value):
        if isinstance(value, bool):
            instance.__dict__[self._name] = value 
        elif isinstance(value, int) and (value == 0 or value == 1):
            instance.__dict__[self._name] = bool(value)
        else:
            raise ValueError("Неправильный булевый тип {}".format(self._name))
            
    def __get__(self, instance, owner):
        return instance.__dict__[self._name]    

In [8]:
class ReadableEntity:
    
    _page_format = ValueFromSet("формат страниц", PAGES_FORMAT.keys())
    _num_pages = NonNegative("число страниц")
    _genre = ValueFromSet("жанр", GENRES)
    _rarity_ind = IntegerBounded("индекс редкости", 1, MAX_RARITY_IND+1)
    _release_date = DatetimeType("дата выхода")
    
    def __init__(self, **kwargs):
        self._name = kwargs['name']
        self._author = kwargs['author']
        self._genre = kwargs['genre']
        self._page_format = kwargs['page_format']
        self._num_pages = kwargs['num_pages']
        self._rarity_ind = kwargs['rarity_ind']
        self._text = kwargs['text']
        self._release_date = kwargs['release_date']
    
    def get_pages_area(self):
        page_size = PAGES_FORMAT[self._page_format]
        return self._num_pages * page_size[0] * page_size[1]

    
class Journal(ReadableEntity):
    _num_release = NonNegative("номер выпуска")
    _num_cosmetic_samples_inside = NonNegative("количество косметических пробников внутри")
    
    def __init__(self, **kwargs):
        
        self._num_release = kwargs['num_release']
        self._num_cosmetic_samples_inside = kwargs['num_cosmetic_samples_inside']
        
        super(Journal, self).__init__(**kwargs)
        
        
class Book(ReadableEntity):
    _has_hard_cover = BooleanType("в твердой обложке")
    _print_date = DatetimeType("дата печати издания книги")
    
    def __init__(self, **kwargs):
        self._has_hard_cover = kwargs['has_hard_cover']
        self._print_date = kwargs['print_date']
        super(Book, self).__init__(**kwargs)
        

class Exporter:
    
    def export_to_txt(self, file_path):
        with open(file_path, 'w') as f:
            for key in self.__dict__:
                f.write("{}: {}".format(key, self.__dict__[key]))
     
    
class LibraryJournal(Journal, Exporter):
    def __init__(self, **kwargs):
        if kwargs['rarity_ind'] >= 9:
            self.export_to_txt(os.path.join("rare_journals", "{}.txt".format(kwargs[name]))) 
        super(LibraryJournal, self).__init__(**kwargs)
    


class LibraryBook(Book, Exporter):
    def __init__(self, **kwargs):
        if kwargs['rarity_ind'] >= 9:
            self.export_to_txt(os.path.join("rare_books", "{}.txt".format(kwargs[name]))) 
        super(LibraryBook, self).__init__(**kwargs)

In [11]:
book1 = LibraryBook(name="Book1",
                    author="Author",
                    genre="Роман",
                    page_format="A1",
                    num_pages=10,
                    rarity_ind=1,
                    text="text",
                    release_date=datetime.date(2010, 10, 1),
                    has_hard_cover=False,
                    print_date=datetime.date(2015, 6, 5),
                    num_cosmetic_samples_inside=10,
                    )

In [12]:
book1.get_pages_area()

20971520

In [13]:
journal1 = LibraryJournal(name="LibraryJournal",
                    author="Author",
                    genre="Драма",
                    page_format="A1",
                    num_pages=100,
                    rarity_ind=1,
                    text="text_library_journal",
                    release_date=datetime.date(2010, 10, 1),
                    has_hard_cover=False,
                    num_release=25,
                    num_cosmetic_samples_inside=5)

In [14]:
journal1.get_pages_area()

209715200

In [15]:
try:
    journal1 = LibraryJournal(name="LibraryJournal",
                        author="Author",
                        genre="Жанр",
                        page_format="A1",
                        num_pages=100,
                        rarity_ind=1,
                        text="text_library_journal",
                        release_date=datetime.date(2010, 10, 1),
                        has_hard_cover=False,
                        num_release=25,
                        num_cosmetic_samples_inside=5)
except Exception as e:
    print(e)

Неизвестное значение для жанр: Жанр, должно быть одно из: Роман, Приключение, Драма


### Задача 1 [Размер объектов] (0 - 0.15 балла)  

**Условие:** 

Написать функцию получения реального объема занимаемой объектом памяти объектом. 


1) Для int, str, list, tuple, dict **(0.05 балла)**

2) Для всех типов **(+0.1 балла)**


**Комментарий:**

На занятиях не раз говорилось, что `sys.getsizeof` умеет находить размер простых объектов, но если речь идет об объектах, вроде list, то функция вернет не совсем то, что может ожидать разработчик, потому что список хранит указатели на объекты. 

*Пример:*
```
sys.getsizeof([]) == 64
sys.getsizeof(['aaaaaaa']) == 72
```
Но
```
sys.getsizeof('aaaaaaa') == 56
```


In [16]:
import sys

def get_size_of(obj, seen=None):
    def get_size_of_dict(d):
        size = 0
        
        size += sum([get_size_of(v) for v in d.values()])
        size += sum([get_size_of(k) for k in d.keys()])
        
        return size
        #return sum([get_size_of(t) for t in d.items()])
    
    if seen is None:
        seen = set()
        
    _id = id(obj)
    if _id in seen:
        return 0
    seen.add(_id)
    
    size = sys.getsizeof(obj)
    
    if isinstance(obj, dict):
        size += get_size_of_dict(obj)
    elif hasattr(obj, '__dict__'):
        size += get_size_of_dict(obj.__dict__)
    elif hasattr(obj, '__iter__') and not isinstance(obj, (str, bytes, bytearray)):
        size += sum([get_size_of(i) for i in obj])
        
    return size

In [17]:
class Room:
    def __init__(self, l, w, h, table):
        self.lenght = l
        self.width = w
        self.height = h
        self.room = table    

class Table:
    def __init__(self, l, w, h):
        self.lenght = l
        self.width = w
        self.height = h

In [18]:
print("My get_size_of vs python's getsizeof")
print("Int:\t{} vs {}".format(get_size_of(1234567890), sys.getsizeof(1234567890)))
print("Float:\t{} vs {}".format(get_size_of(1234567890.1), sys.getsizeof(1234567890.1)))
print("String:\t{} vs {}".format(get_size_of('abc'), sys.getsizeof('abc')))
print("Tuple:\t{} vs {}".format(get_size_of(('abc', 1)), sys.getsizeof(('abc', 1))))
print("Empty list:\t{} vs {}".format(get_size_of([]), sys.getsizeof([])))
print("List with 2 strings:\t{} vs {}".format(get_size_of(['abc', 'qwe']), sys.getsizeof(['abc', 'qwe'])))
print("Dict:\t{} vs {}".format(get_size_of({'abc' : 1, 'qwe' : 2}), sys.getsizeof({'abc' : 1, 'qwe' : 2})))
room = Room(1, 2, 3, Table(4, 5, 6))
print("Class with fields:\t{} vs {}".format(get_size_of(room), sys.getsizeof(room)))

My get_size_of vs python's getsizeof
Int:	32 vs 32
Float:	24 vs 24
String:	52 vs 52
Tuple:	144 vs 64
Empty list:	64 vs 64
List with 2 strings:	184 vs 80
Dict:	400 vs 240
Class with fields:	661 vs 56


### Задача 2 [Многочлены] (0.64 балла)

**Условие:**

Реализовать класс многочлена. Определить операции:

1) *сложения* - **(0.02 балла)**

2) *вычитания* - **(0.02 балла)**

3) *умножения* - **(0.04 балла)**

3a) *быстрого умножения* (алгоритм Карацубы или быстрое преобразование Фурье) - **(+0.25 балла)**

4) *деления* - **(0.05 балла)**

5) *возведения в степень* - **(0.02 балла)** | *возведения в степень* через быстрое возведение в степень за log - **(0.04 балла)**

6) *представления многочлена в человеческом виде* - **(0.02 балла)**

7) *дифференцирования* - **(0.05 балла)**

8) *интегрирования* - **(0.05 балла)**

9) Вызова многочлена как функции (вычисление значения в точке) - **(0.03 балла)**

**Комментарии:**

Для комплексных коэффициентов **(0.01 балла)** к каждому пункту.

Операции с числами также должны работать.

In [19]:
def foo(*l):
    print(l)

In [20]:
foo([1,2,3])

([1, 2, 3],)


In [21]:
coefs=[1,2,3]
print("+".join(["{}x^{}".format(coef, coef) for coef in coefs[::-1]]))

3x^3+2x^2+1x^1


In [22]:
from itertools import zip_longest
from copy import copy

class Polynomial:
    
    def __init__(self, *coefs):
        if len(coefs) == 0:
            coefs = [0]
        self.coefs = list(coefs)[::-1] #logic from low to high
        self.trim()
        
    def __call__(self, x):    
        return sum([coef * x ** i for i, coef in enumerate(self.coefs)])
    
    def degree(self):
        return len(self.coefs) - 1   
            
    def __add__(self, other):
        if isinstance(other, Polynomial):
            res = [sum(t) for t in zip_longest(self.coefs, other.coefs, fillvalue=0)]
            
        else:
            res = copy(self.coefs)
            res[0] += other
        return Polynomial(*res[::-1])
    
    def __radd__(self, other):
        return self.__add__(other)
    
    def __sub__(self, other):
        if isinstance(other, Polynomial):
            res = [t1-t2 for t1, t2 in zip_longest(self.coefs, other.coefs, fillvalue=0)]
        else:
            res = copy(self.coefs)
            res[0] -= other
        return Polynomial(*res[::-1])
    
    def __rsub__(self, other):
        return self.__sub__(other)
    
    def __mul__(self, other):
        if isinstance(other, Polynomial):
            res = [0] * (len(self.coefs) + len(other.coefs) - 1)
            for i in range(len(self.coefs)):
                for j in range(len(other.coefs)):
                    res[i+j] += self.coefs[i] * self.coefs[j]
        else:
            res = [other * coef for coef in self.coefs]           
        return Polynomial(*res[::-1])
                                     
    def __rmul__(self, other):
        return self.__mul__(other)
    
    def derivate(self):
        derived_coeffs = [i * self.coefs[i] for i in range(len(self.coefs))][1:]
        return Polynomial(*derived_coeffs[::-1])
    
    def integrate(self, const=0):
        integrated_coeffs = [const] + [1/(i+1) * self.coefs[i] for i in range(len(self.coefs))]
        return Polynomial(*integrated_coeffs[::-1])
    
    def trim(self):
        zeros_cnt = 0
        for i in range(len(coefs)-1,-1,-1):
            if coefs[i] == 0:
                zeros_cnt += 1
            else:
                break
        del self.coefs[len(self.coefs) - zeros_cnt:]
        
    def __str__(self):
        parts = []
        result = ""
        for i, coef in enumerate(self.coefs[::-1]):
            power = len(self.coefs) - 1 - i
            if coef == 0:
                continue

            sign = "+" if coef > 0 else "-"

            if sign == "+" and len(result) == 0:
                sign = ""

            if power == 0:
                print_power = ""
            elif power == 1: 
                print_power = "x"
            else:
                print_power = "x^" + str(power)


            if abs(coef) != 1 or print_power == "":
                print_coef = str(abs(coef))
            else:
                print_coef = ""

            result += sign + print_coef + print_power
        if result == "":
            return "0"
        return result

In [27]:
p1 = Polynomial(2, 1)
p2 = Polynomial(2, 1)
print(p1)
print(p1(10))
print(p2)

print("-add-")
print(p1+p2)
print(10+p1)
print(p1+10)

print("-sub-")
print(p1-p2)
print(10-p1)
print(p1-10)

print("-multiply-")
print(p1*p2)
print(10*p1)
print(p1*10)



2x+1
21
2x+1
-add-
4x+2
2x+11
2x+11
-sub-
0
2x-9
2x-9
-multiply-
4x^2+4x+1
20x+10
20x+10


### Задача 3 [Аналог range] (0.05 балла)

**Условие:**

Реализуйте итератор с поведением, аналогичным range.

In [39]:
def my_range(start, stop=None, step=1):
    if step == 0:
        raise ValueError('my_range() arg 3 must not be zero')
    if not isinstance(start, int) or not isinstance(start, int) or not isinstance(step, int):
        raise TypeError('args must be integer')
    
    if stop is None:
        stop = start
        start = 0
    
    if start < stop and step > 0:
        while start < stop:
            yield start
            start += step
    elif stop < start and step < 0:
        while stop < start:
            yield start
            start += step

start = 30
stop = 10
step = -1

print([i for i in my_range(start, stop, step)])
print([i for i in range(start, stop, step)])

start = 10
stop = 30
step = 1

print([i for i in my_range(start, stop, step)])
print([i for i in range(start, stop, step)])

[30, 29, 28, 27, 26, 25, 24, 23, 22, 21, 20, 19, 18, 17, 16, 15, 14, 13, 12, 11]
[30, 29, 28, 27, 26, 25, 24, 23, 22, 21, 20, 19, 18, 17, 16, 15, 14, 13, 12, 11]
[10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29]
[10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29]


### Задача 4 [Primary Key] (0.05 балла)

**Условие:**

С помощью механизма дескрипторов реализуйте Primary Key - свойства первичного ключа из PostgreSQL.

In [29]:
# Ограничение на PrimaryKey:
# 1) Уникальный
# 2) Not null
# 3) Всего 1 PrimaryKey на класс

class PrimaryKey:
    
    def __init__(self, name):
        self.name = name 
    
    def __get__(self, instance, owner):
        return instance.__dict__[self.name]  
    
    def __set__(self, instance, value):
        if self.has_sevaral_pk(instance, value):
            raise TypeError('Должно быть одно поле Primary Key ')
            
        if value is None:
            raise ValueError('Primary Key не может быть None')

        if not self.is_unique(instance, value):
            raise ValueError('Primary Key должен быть уникальным')
            
        instance.__dict__[self.name] = value
    
    def is_unique(self, instance, value):
        if not '_existing_pk' in instance.__class__.__dict__ or instance.__class__._existing_pk is None:
            instance.__class__._existing_pk = set()
        result = not value in instance.__class__._existing_pk
        instance.__class__._existing_pk.add(value)
        return result
    
    def has_sevaral_pk(self, instance, value):
        if '_already_has_pk' in instance.__dict__ and instance._already_has_pk:
            return True
        instance._already_has_pk = True
        return False

In [30]:
class PkExample1:
    number = PrimaryKey('number') 
   
    def __init__(self, number):
        self.number = number

i1 = PkExample1(1)
i2 = PkExample1(2)
i3 = PkExample1(3)
i3 = PkExample1('a')
i3 = PkExample1('abc')

print("Everything is fine")

Everything is fine


In [31]:
class PkExample2:
    number = PrimaryKey('number') 
   
    def __init__(self, number):
        self.number = number
try:
    i1 = PkExample2(1)
    i1 = PkExample2(1)
except Exception as e:
    print(e)

Primary Key должен быть уникальным


In [32]:
class PkExample3:
    number = PrimaryKey('number') 
   
    def __init__(self, number):
        self.number = number
try:
    i1 = PkExample3(None)
except Exception as e:
    print(e)

Primary Key не может быть None


In [33]:
class PkExample4:
    number1 = PrimaryKey('number1') 
    number2 = PrimaryKey('number2') 
   
    def __init__(self, number1, number2):
        self.number1 = number1
        self.number2 = number2
try:
    i1 = PkExample4(1, 2)
except Exception as e:
    print(e)

Должно быть одно поле Primary Key 


### Задача 5 [PositiveSmallIntegerField] (0.03 балла)

**Условие:**

С помощью механизма дескрипторов реализуйте тип данных PositiveSmallIntegerField - поле, принимающее значения от 0 до 32767.

In [34]:
class PositiveSmallIntegerField:
    def __init__(self, name):
        self.name = name 
    
    def __get__(self, instance, owner):
        return instance.__dict__[self.name]  
    
    def __set__(self, instance, value):
        if value < 0 or value > 32767:
            raise ValueError('Не может быть меньше 0 и больше 32767')
        instance.__dict__[self.name] = value

class Example:
    number = PositiveSmallIntegerField('number') 
   
    def __init__(self, number):
        self.number = number

In [35]:
example = Example(5)

In [36]:
try:
    example = Example(-1)
except Exception as e:
    print(e)

Не может быть меньше 0 и больше 32767


In [37]:
try:
    example = Example(32768)
except Exception as e:
    print(e)

Не может быть меньше 0 и больше 32767


### Задача 6 [Timer] (0.02 балла)

**Условие:**

Реализовать контекстный менеджер, который выводит время, проведенное в нём.

In [38]:
import datetime
import time

class TimerContextManager:
    
    def __enter__(self):
        self.enter_time = datetime.datetime.now()

    def __exit__(self, exc_type, exc_val, exc_tb):
        interval = datetime.datetime.now() - self.enter_time
        print(f'Time in context manager: {interval}')
        

with TimerContextManager():
    print("Enter in TimerContextManager")
    time.sleep(3)

print("Exit from TimerContextManager")

Enter in TimerContextManager
Time in context manager: 0:00:03.000592
Exit from TimerContextManager
